<a href="https://colab.research.google.com/github/Wilson30678tw/IAT460_A2/blob/main/Lin_Che_wei_rulebased_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# First, let's import our required libraries
from collections import defaultdict
import random

In [42]:
# 定義故事語法
story_grammar = {
    'STORY': [['INTRO', 'CONFLICT', 'RESOLUTION']],  # Full story structure
    'INTRO': [['CHARACTER', 'ACTION', 'PLACE']],  # Introduction
    'CONFLICT': [['ENEMY', 'MOTIVATION', 'PROBLEM']],  # Conflict structure
    'RESOLUTION': [['ALLY', 'ABILITY', 'OUTCOME']],  # Resolution

    # Characters & Places
    'CHARACTER': ['a wandering knight', 'a lost sorcerer', 'an elf archer', 'a rogue thief'],
    'PLACE': ['in the ancient ruins', 'within the cursed forest', 'beyond the forgotten kingdom', 'under the dragon’s lair'],

    # Actions & Problems
    'ACTION': ['discovered', 'was searching for', 'was ambushed in', 'encountered'],
    'PROBLEM': ['a sealed portal', 'a forbidden spellbook', 'a hidden artifact', 'a prophecy that foretells doom'],

    # Enemy & Motivation
    'ENEMY': ['a dark sorcerer', 'a wicked warlord', 'a cursed dragon', 'an ancient demon'],
    'MOTIVATION': ['seeks to conquer the kingdom', 'plans to steal the artifact', 'desires to break an ancient curse', 'aims to overthrow the gods'],

    # Ally, Ability & Outcome
    'ALLY': ['a wise old mage', 'a cunning rogue', 'a celestial spirit', 'a talking owl'],
    'ABILITY': ['used a powerful spell', 'unleashed elemental magic', 'discovered a hidden passage', 'activated an ancient relic'],
    'OUTCOME': ['and banished the darkness', 'and sealed the enemy away forever', 'and restored balance to the world', 'and reclaimed the lost treasure']
}

# # CFG 生成函數
# def generate_text(symbol, grammar):
#     if isinstance(symbol, str) and symbol in grammar:
#         production = random.choice(grammar[symbol])
#         if isinstance(production, list):
#             return ' '.join(generate_text(sym, grammar) for sym in production)
#         return production
#     return symbol

# def generate_story():
#     return generate_text('STORY', story_grammar)



Markov Model in here

In [43]:
def build_markov_chain(text, order=2):

    chain = defaultdict(list)
    words = text.split()

    for i in range(len(words) - order):
        # Create state tuple from current words
        state = tuple(words[i:i+order])
        # Get the next word
        next_word = words[i+order]
        # Add to chain
        chain[state].append(next_word)

    return chain

def generate_markov_text(chain, num_words=80):
    """
    使用 Markov Chain 生成故事擴展敘述
    """
    words = list(random.choice(list(chain.keys())))
    order = len(list(chain.keys())[0])

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state in chain:
            next_word = random.choice(chain[state])
            words.append(next_word)
        else:
            break

    return ' '.join(words)


In [44]:
sample_text = """
In an ancient kingdom, a wandering knight set forth on a perilous journey to uncover a forgotten prophecy.
The knight traveled through shadowed valleys and across mist-covered mountains, searching for the lost relic.
Legends spoke of an enchanted portal hidden deep within the cursed forest, a gateway said to hold unimaginable power.
Many had sought the portal, yet none had returned to tell its secrets.
One fateful night, the knight discovered an ancient tablet inscribed with cryptic runes, revealing the portal’s location.
Following the markings, the knight ventured into the depths of the ruins, where a chilling presence loomed in the darkness.
A menacing growl echoed through the halls, and the air grew heavy with magic.
The knight's hand tightened around their blade as a dark figure emerged from the shadows.
Before them stood a sorcerer, cloaked in midnight, seeking to claim the relic for his own.
With the fate of the kingdom hanging in the balance, the knight prepared to face their greatest challenge.
"""

# 建立 Markov Chain
markov_chain = build_markov_chain(sample_text, order=3)

def generate_full_story():

    structured_story = generate_story()  # 產生基本故事架構
    dynamic_text = generate_markov_text(markov_chain, num_words=80)  # 產生隨機敘述

    # **確保總字數至少 100**
    full_story = f"{structured_story}\n\n{dynamic_text}"
    word_count = len(full_story.split())

    while word_count < 100:
        extra_text = generate_markov_text(markov_chain, num_words=20)
        full_story += f" {extra_text}"
        word_count = len(full_story.split())

    return full_story

In [45]:
print(generate_full_story())

a rogue thief encountered beyond the forgotten kingdom a wicked warlord desires to break an ancient curse a hidden artifact a talking owl discovered a hidden passage and banished the darkness

knight prepared to face their greatest challenge. with magic. The knight's hand tightened around their blade as a dark figure emerged from the shadows. Before them stood for the lost relic. Legends spoke of an enchanted portal hidden deep within the cursed forest, a gateway said to The knight traveled through shadowed valleys and across mist-covered mountains, searching for the lost relic. Legends spoke of an enchanted inscribed with cryptic runes, revealing the portal’s location. Following the markings, the knight ventured into the depths of the ruins,
